In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
tf.set_random_seed(777)

In [4]:
# parameters
log_dir = './logs/ML_Lab_11-08-Deep_CNN_class'
learning_rate = 0.001
batch_size = 100
num_epochs = 50

In [5]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [6]:
class Model:
    def __init__(self, sess, name, learning_rate=0.001):
        self.sess = sess
        self.name = name
        self.learning_rate = learning_rate
        self._build_net()
    def _build_net(self):
        self.dropout_rate = tf.placeholder(shape=None, dtype=tf.float32)
        self.X = tf.placeholder(shape=[None, 784], dtype=tf.float32, name='X')
        self.Y = tf.placeholder(shape=[None,  10], dtype=tf.float32, name='Y')
        X_img = tf.reshape(self.X, shape=[-1, 28, 28, 1])
        with tf.variable_scope('Deep-CNN_' + self.name) as scope:
            with tf.variable_scope('Conv1') as scope:
                W1 = tf.Variable(tf.random_normal(shape=[3, 3, 1, 32], stddev=0.01), dtype=tf.float32, name='weight1')
                L1 = tf.nn.conv2d(X_img, W1, strides=[1, 1, 1, 1], padding='SAME')
                L1 = tf.nn.relu(L1)
                L1 = tf.nn.max_pool(L1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                L1 = tf.nn.dropout(L1, rate=self.dropout_rate)
                print(L1)
            with tf.variable_scope('Conv2') as scope:
                W2 = tf.Variable(tf.random_normal(shape=[3, 3, 32, 64], stddev=0.01), dtype=tf.float32, name='weight2')
                L2 = tf.nn.conv2d(L1, W2, strides=[1, 1, 1, 1], padding='SAME')
                L2 = tf.nn.relu(L2)
                L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                L2 = tf.nn.dropout(L2, rate=self.dropout_rate)
                print(L2)
            with tf.variable_scope('Conv3') as scope:
                W3 = tf.Variable(tf.random_normal(shape=[3, 3, 64, 128], stddev=0.01), dtype=tf.float32, name='weight3')
                L3 = tf.nn.conv2d(L2, W3, strides=[1, 1, 1, 1], padding='SAME')
                L3 = tf.nn.relu(L3)
                L3 = tf.nn.max_pool(L3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
                L3 = tf.nn.dropout(L3, rate=self.dropout_rate)
                print(L3)
            with tf.variable_scope('Fuuly-connected-1') as scope:
                L3 = tf.reshape(L3, shape=[-1, 4 * 4 * 128])
                W4 = tf.get_variable('weight4', shape=[4 * 4 * 128, 625], initializer=tf.contrib.layers.xavier_initializer())
                b4 = tf.Variable(tf.random_normal(shape=[625]), dtype=tf.float32, name='bias4')
                L4 = tf.matmul(L3, W4) + b4
                L4 = tf.nn.relu(L4)
                L4 = tf.nn.dropout(L4, rate=self.dropout_rate)
                print(L4)
            with tf.variable_scope('Fuuly-connected-2') as scope:
                W5 = tf.get_variable('weight5', shape=[625, 10], initializer=tf.contrib.layers.xavier_initializer())
                b5 = tf.Variable(tf.random_normal(shape=[10]), dtype=tf.float32, name='bias5')
                self.logits = tf.matmul(L4, W5) + b5
                print(self.logits)
        
            with tf.variable_scope('cost'):
                self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.Y))

            with tf.variable_scope('Optimizer'):
                print(self.learning_rate)
                self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

            with tf.variable_scope('accuracy') as scope:
                is_correct = tf.equal(tf.argmax(self.logits, 1), tf.arg_max(self.Y, 1))
                self.accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
    def predict(self, x_test, dropout_rate=0.0):
        return sess.run(self.logits, feed_dict={self.X: x_test, self.dropout_rate: dropout_rate})

    def get_accuracy(self, x_test, y_test, dropout_rate=0.0 ):
        return sess.run(self.accuracy, feed_dict={self.X: x_test, self.y: y_test, self.dropout_rate: dropout_rate})
    
    def train(self, x_data, y_data, dropout_rate=0.3):
        return sess.run([self.cost, self.optimizer], feed_dict={self.X: x_data, self.Y: y_data, self.dropout_rate: dropout_rate})[0]
        

In [7]:
with tf.Session() as sess:
    m1 = Model(sess, 'm1')
    tf.summary.scalar('cost', m1.cost)
    tf.summary.scalar('accuracy', m1.accuracy)
    
    # tensorboard --logdir=./logs/Deep_NN
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter(log_dir)
    writer.add_graph(sess.graph)
    
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
    # Training cycle
    print('Learning stared. It takes sometime.')
    for epoch in range(num_epochs):
        avg_cost = 0
        total_batch = int(mnist.train.num_examples / batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            m1.train(batch_xs,batch_ys, dropout_rate=0.3)
            c_val = m1.train(batch_xs,batch_ys, dropout_rate=0.3)
            avg_cost += c_val / total_batch
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))
        print('Accuracy: ', m1.get_accuracy(mnist.test.images, mnist.test.labels, learning_rate=0.001, dropout_rate=0.0))
        summary = sess.run(merged_summary, feed_dict={X:mnist.test.images, Y: mnist.test.labels, dropout_rate: 0.})
        writer.add_summary(summary, global_step=(epoch))
    print('Learning Finished!')

Tensor("Deep-CNN_m1/Conv1/dropout/mul_1:0", shape=(?, 14, 14, 32), dtype=float32)
Tensor("Deep-CNN_m1/Conv2/dropout/mul_1:0", shape=(?, 7, 7, 64), dtype=float32)
Tensor("Deep-CNN_m1/Conv3/dropout/mul_1:0", shape=(?, 4, 4, 128), dtype=float32)
Tensor("Deep-CNN_m1/Fuuly-connected-1/dropout/mul_1:0", shape=(?, 625), dtype=float32)
Tensor("Deep-CNN_m1/Fuuly-connected-2/add:0", shape=(?, 10), dtype=float32)
0.001
Instructions for updating:
Use `tf.math.argmax` instead
Learning stared. It takes sometime.


AttributeError: 'Model' object has no attribute 'dropout_rate'